In [ ]:
from datetime import date
import calendar
from dateutil.relativedelta import relativedelta
from scipy.optimize import fsolve
import pandas as pd
import math

def pmt(intrestRate, loanDuration, principal):
    factor = (1 + intrestRate) ** loanDuration
    return intrestRate * principal * factor / (factor - 1)

loan_amount = 2700

interest_rate = 0.13
loan_duration = 35
today = date.today() #date(2025, 8, 15)


purchase_month_inerest = loan_amount * interest_rate * (30-today.day+1) / 360
principal_with_interest = loan_amount + purchase_month_inerest
monthly = math.ceil(-pmt(interest_rate/12, loan_duration, -principal_with_interest))

purchase_interest_total = math.ceil(loan_amount * interest_rate * (30-today.day+1+30) / 360)

opening_balances = []
closing_balances = []
interests = []
purchase_interests = []
monthly_payments = []

for i in range(1, loan_duration + 1):
    opening_balance = loan_amount if i == 1 else closing_balances[-1]
    opening_balances.append(opening_balance)

    # Purchase interest is paid on second month
    purchase_interest = purchase_interest_total if i == 2 else 0.0
    purchase_interests.append(purchase_interest)

    interest = 0 if i == 1 else math.ceil((opening_balance + purchase_interest) * interest_rate / 12)
    interests.append(interest)

    balance_with_interest = opening_balance + purchase_interest + interest
    # paying monthly payment or what's left
    payment = monthly if monthly < balance_with_interest else balance_with_interest
    monthly_payments.append(payment)
    closing_balances.append(balance_with_interest - payment)

payment_plan = pd.DataFrame({
    "Month": range(1, loan_duration + 1),
    "Opening Balance": opening_balances,
    "Purchase Interest": purchase_interests,
    "Interest": interests,
    "Payment": monthly_payments,
    "Closing Balance": closing_balances
})

print(payment_plan)

Monthly payment: 94
Total purchase interest: 40
    Month  Opening Balance  Purchase Interest  Interest  Payment  \
0       1           2700.0                0.0         0     94.0   
1       2           2606.0               40.0        29     94.0   
2       3           2581.0                0.0        28     94.0   
3       4           2515.0                0.0        28     94.0   
4       5           2449.0                0.0        27     94.0   
5       6           2382.0                0.0        26     94.0   
6       7           2314.0                0.0        26     94.0   
7       8           2246.0                0.0        25     94.0   
8       9           2177.0                0.0        24     94.0   
9      10           2107.0                0.0        23     94.0   
10     11           2036.0                0.0        23     94.0   
11     12           1965.0                0.0        22     94.0   
12     13           1893.0                0.0        21     94.0   


In [ ]:
def end_of_month(start_date, months):
    """excel eomonth function"""
    new_date = start_date + relativedelta(months=months)

    _, last_day = calendar.monthrange(new_date.year, new_date.month)

    return date(new_date.year, new_date.month, last_day)

cash_flows = [-loan_amount] + monthly_payments

dates = [today] + [end_of_month(today, i) for i in range(1, len(monthly_payments) + 1)]
days = [(dt - today).days for dt in dates]

table = pd.DataFrame({
    'Cash_Flow': cash_flows,
    'Description': ['Initial Loan'] + ['Monthly Payment'] * (loan_duration - 1) + ['Final Payment'],
    'Payment dates': dates,
    'Days from debit': days
})

print(table)

    Cash_Flow      Description Payment dates  Days from debit
0     -2700.0     Initial Loan    2025-08-20                0
1        94.0  Monthly Payment    2025-09-30               41
2        94.0  Monthly Payment    2025-10-31               72
3        94.0  Monthly Payment    2025-11-30              102
4        94.0  Monthly Payment    2025-12-31              133
5        94.0  Monthly Payment    2026-01-31              164
6        94.0  Monthly Payment    2026-02-28              192
7        94.0  Monthly Payment    2026-03-31              223
8        94.0  Monthly Payment    2026-04-30              253
9        94.0  Monthly Payment    2026-05-31              284
10       94.0  Monthly Payment    2026-06-30              314
11       94.0  Monthly Payment    2026-07-31              345
12       94.0  Monthly Payment    2026-08-31              376
13       94.0  Monthly Payment    2026-09-30              406
14       94.0  Monthly Payment    2026-10-31              437
15      

In [87]:
def npv(rate, cash_flows, days):
    """Calculate Net Present Value for given rate"""
    return sum(cf * (1 + rate) ** (-d / 365) for _, (cf, d) in enumerate(zip(cash_flows, days)))

def calculate_irr_scipy(cash_flows, dates, initial_guess=0.01):
    """Calculate IRR using scipy.optimize.fsolve"""
    try:
        irr = fsolve(lambda r: npv(r, cash_flows, dates), initial_guess)[0]
        return irr
    except:
        return None
    
monthly_irr = calculate_irr_scipy(cash_flows, days)
print(f"Monthly IRR (using scipy): {monthly_irr:.4f}")

Monthly IRR (using scipy): 0.1417
